In [1]:
load('../sim_data/independent_Lasso.RData')

In [2]:
dir = '../sim_data'
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [3]:
avg_FDR = NULL
for (i in 1:length(files)){
    sim_file = files[i]
    load(sim_file, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_lasso[[i]]$Stab.table
    
    FDR = NULL
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}


In [16]:
table_ind = NULL
tmp_num_select = rep(0, length(results_ind_lasso))
for (i in 1:length(results_ind_lasso)){
    table_ind = rbind(table_ind, results_ind_lasso[[i]][c('n', 'p', 'rou', 'FP', 'FN', 'MSE', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_ind_lasso[[i]]$Stab.table))
    
}
table_ind = as.data.frame(table_ind)
table_ind$num_select = tmp_num_select
table_ind$FDR = round(avg_FDR,2)

In [17]:
head(table_ind)

n,p,rou,FP,FN,MSE,Stab,num_select,FDR
50,50,0,8.36 ( 0.45 ),0.02 ( 0.01 ),0.54 ( 0.03 ),0.37,13.34,0.51
100,50,0,5.5 ( 0.42 ),0 ( 0 ),0.36 ( 0.01 ),0.51,10.50,0.37
500,50,0,2.33 ( 0.14 ),0 ( 0 ),0.28 ( 0 ),0.79,7.33,0.15
1000,50,0,1.82 ( 0.13 ),0 ( 0 ),0.26 ( 0 ),0.86,6.82,0.10
50,100,0,11.1 ( 0.38 ),0.08 ( 0.03 ),0.66 ( 0.04 ),0.32,16.02,0.61
100,100,0,7.23 ( 0.4 ),0 ( 0 ),0.41 ( 0.01 ),0.46,12.23,0.46


In [18]:
# export result
result.table_ind <- apply(table_ind,2,as.character)
rownames(result.table_ind) = rownames(table_ind)
result.table_ind = as.data.frame(result.table_ind)

# extract numbers only for 'n' & 'p'
result.table_ind$n = tidyr::extract_numeric(result.table_ind$n)
result.table_ind$p = tidyr::extract_numeric(result.table_ind$p)
result.table_ind$ratio = result.table_ind$p / result.table_ind$n

result.table_ind = result.table_ind[c('n', 'p', 'ratio', 'Stab', 'MSE', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_ind)[1:3] = c('N', 'P', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead
extract_numeric() is deprecated: please use readr::parse_number() instead


In [19]:
# convert interested measurements to be numeric
result.table_ind$Stab = as.numeric(as.character(result.table_ind$Stab))
result.table_ind$MSE_mean = as.numeric(substr(result.table_ind$MSE, start=1, stop=4))
result.table_ind$FP_mean = as.numeric(substr(result.table_ind$FP, start=1, stop=4))
result.table_ind$FN_mean = as.numeric(substr(result.table_ind$FN, start=1, stop=4))
result.table_ind$FN_mean[is.na(result.table_ind$FN_mean)] = 0
result.table_ind$num_select = as.numeric(as.character(result.table_ind$num_select))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [21]:
result.table_ind

## export
write.table(result.table_ind, '../results_summary/sim_ind_lasso.txt', sep='\t', row.names=F)

N,P,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
50,50,1.00,0.37,0.54 ( 0.03 ),8.36 ( 0.45 ),0.02 ( 0.01 ),13.34,0.51,0.54,8.36,0.02
100,50,0.50,0.51,0.36 ( 0.01 ),5.5 ( 0.42 ),0 ( 0 ),10.50,0.37,0.36,5.50,0.00
500,50,0.10,0.79,0.28 ( 0 ),2.33 ( 0.14 ),0 ( 0 ),7.33,0.15,0.28,2.33,0.00
1000,50,0.05,0.86,0.26 ( 0 ),1.82 ( 0.13 ),0 ( 0 ),6.82,0.1,0.26,1.82,0.00
50,100,2.00,0.32,0.66 ( 0.04 ),11.1 ( 0.38 ),0.08 ( 0.03 ),16.02,0.61,0.66,11.10,0.08
100,100,1.00,0.46,0.41 ( 0.01 ),7.23 ( 0.4 ),0 ( 0 ),12.23,0.46,0.41,7.23,0.00
500,100,0.20,0.71,0.29 ( 0 ),3.26 ( 0.25 ),0 ( 0 ),8.26,0.22,0.29,3.26,0.00
1000,100,0.10,0.82,0.27 ( 0 ),2.25 ( 0.15 ),0 ( 0 ),7.25,0.15,0.27,2.25,0.00
50,500,10.00,0.19,1.41 ( 0.11 ),19.89 ( 0.38 ),0.64 ( 0.08 ),24.25,0.77,1.41,19.80,0.64
100,500,5.00,0.30,0.48 ( 0.02 ),14.71 ( 0.7 ),0 ( 0 ),19.71,0.66,0.48,14.70,0.00


In [10]:
library(ggplot2)
library(gridExtra)
pdf('../figures_sim/figure_independent_lasso.pdf', height=5, width=7)
result.table_ind$N = as.factor(result.table_ind$N)
fig_ind_stab = ggplot(result.table_ind, aes(x=P, y=Stab, color=N)) + geom_point() + ylab('Stability')
fig_ind_mse = ggplot(result.table_ind, aes(x=P, y=MSE_mean, color=N)) + geom_point() + ylab('MSE')
fig_ind_fp = ggplot(result.table_ind, aes(x=P, y=FP_mean, color=N)) + geom_point() + ylab('False Positives')
fig_ind_fn = ggplot(result.table_ind, aes(x=P, y=FN_mean, color=N)) + geom_point() + ylab('False Negatives')
grid.arrange(fig_ind_stab, fig_ind_mse, fig_ind_fp, fig_ind_fn, ncol=2, top='Independent_Lasso')
dev.off()

pdf 
  2